# Naive Bayes 

In [15]:
import pandas as pd
import numpy as np

## Carga de datos

#### Limpieza de datos
**Nota:** En esta primera versión, la función trata de enfocarse en el dataset de Brast Cancer Wisconsin, pero pretende que se sueda llegar a implementar con otros agregando más argumentos. 

In [6]:
def clean(ds, characters=['?'], col=None):
    for j in col:
        for i in range(ds.shape[0]):
            if ds[i,j] in characters:
                ds[i,j] = np.nan
        ds[:,j] = ds[:,j].astype(np.float64)

        mean = ds[:,j].astype(np.float64).mean(where=[0 if np.isnan(e) else 1 for e in ds[:,j]])

        for i, e in enumerate(ds[:, j]):
            if np.isnan(e):
                ds[i,j] = mean

    return 'Datos limpios exitosamente'

### Iris Plant

In [16]:
data = pd.read_csv('iris.data', header=None).values
label_col = 4 # Obtenemos el indice de la etiqueta
chr_cols = (1, data.shape[1]) if label_col == 0 else (0, data.shape[1]-1) # Obtener las columnas de características

### Breast Cancer

In [14]:
data = pd.read_csv('Breast_cancer_data.csv').sort_values('diagnosis', ignore_index=True).values
label_col = 5 # Columna de la etiqueta
chr_cols = (1, data.shape[1]) if label_col == 0 else (0, data.shape[1]-1) # Obtener las columnas de características

### Breast Cancer Wisconsin

In [7]:
label_col = 10 # Columna de la etiqueta
data = pd.read_csv('breast-cancer-wisconsin.data', header=None).sort_values(label_col, ignore_index=True).values
chr_cols = (1,9) # Obtener las columnas de las características
clean(data, col=[6]) # Se implementa la limpieza de datos

'Datos limpios exitosamente'

### Wine

In [29]:
data = pd.read_csv('wine.data', header=None).values
label_col = 0
chr_cols = (1, data.shape[1]) if label_col == 0 else (0, data.shape[1]-1) # Obtener las columnas de características

## Desarrollo de algoritmo de Bayes

### Teorema de Bayes
$P(Y|X) = \frac{P(X = x_0, x_1, x_2 | Y = y)P(Y = y)}{P(X)}$

### Calcular la probabilidad a priori

In [17]:
def calculate_prior(ds):
    total = sum(map(lambda arr: arr.shape[0], ds))
    return [arr.shape[0]/total for arr in ds]

### Obtener subarreglos por clase

In [18]:
def get_subarrs_from_dataset(array):
    l = []
    c = 0
    for i, e in enumerate(array):
        if e[label_col] != array[c][label_col]:
            l.append(data[c:i])
            c = i
    l.append(data[c:i+1])

    return np.array(l)

### Obtención del modelo

In [19]:
def get_model(ds, chr_cols):
    # Calcular probabilidad a priori
    prior = calculate_prior(ds)

    # Calcular medias y desviaciones estandar por columna y clase
    measures = []
    for cl in ds:
        m = []
        for ch_col in (cl[:, i] for i in range(*chr_cols)):
            m.append({'mean': ch_col.mean(), 'std': ch_col.std()})
        measures.append(m)

    return (prior, measures)

### Calcular la probabilidad
**Nota:** Para la probabilidad se implementó la distribición normal que se enecuentra dentro de la presentación; sin embargo, en Internet se presenta una de manera diferente:
[The Normal Distribution](https://www.thoughtco.com/normal-distribution-bell-curve-formula-3126278)

In [20]:
def calculate_probability(e, prior, measures):
    probability = 1
    for m, ch in zip(measures, e):
        probability *= (1/np.sqrt(2*np.pi*m['std']))*np.exp(-.5*((ch-m['mean'])/m['std'])**2)
    probability *= prior

    return probability

### Clasificar

In [21]:
def classify(subset, prior, measures):
    accuracies = []
    for i, cl in enumerate(subset):
        accuracy = 0
        for e in (e[chr_cols[0]:chr_cols[1]] for e in cl):
            likelihood = []
            for p, m in zip(prior, measures):
                likelihood.append(calculate_probability(e, p, m))
            if np.argmax(likelihood) == i: accuracy += 1

        accuracies.append(accuracy)

    return np.array(accuracies)

### Entrenamiento y prueba

In [24]:
from sklearn.model_selection import train_test_split

subarrs = get_subarrs_from_dataset(data)

train, test = [], []
for arr in subarrs:
    tr, te = train_test_split(arr, test_size=.2, random_state=50)
    train.append(tr)
    test.append(te)
train = np.array(train)
test = np.array(test)
del subarrs, arr, tr, te

# Obtener modelo
model = get_model(train, chr_cols)

# ****** Resultados ******
# Entrenamiento
acc_train = classify(train, *model)

# Prueba
acc_test = classify(test, *model)
# TODO: Checar warning con datasets Breast y Wine

### Muestra de resultados

In [25]:
def show_results(ds, accuracy, label_col, ds_name):
    print('\nResultados con el subjuntunto de', ds_name)
    for t in range(ds.shape[0]):
        print(f'- Clase \'{ds[t][0][label_col]}\': {accuracy[t]}/{ds[t].shape[0]}')
    print(f'Exactitud: {100*sum(accuracy)/sum(map(lambda e: e.shape[0], ds)):.2f}%')

show_results(train, acc_train, label_col, 'entrenamiento')
show_results(test, acc_test, label_col, 'prueba')


Resultados con el subjuntunto de entrenamiento
- Clase 'Iris-setosa': 40/40
- Clase 'Iris-versicolor': 36/40
- Clase 'Iris-virginica': 37/40
Exactitud: 94.17%

Resultados con el subjuntunto de prueba
- Clase 'Iris-setosa': 10/10
- Clase 'Iris-versicolor': 10/10
- Clase 'Iris-virginica': 9/10
Exactitud: 96.67%
